<a href="https://colab.research.google.com/github/JackGraymer/Advanced-GenAI/blob/main/1_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Generative Artificial Intelligence
**Project - Designing a RAG-Based Q&A System for News Retrieval**

**Authors:** Vsevolod Mironov, Pascal Küng, Alvaro Cervan (Group 5)


# Step 1 - Data preparation

**Contribution:** ....

**Goal of this step:** ....

## 1. Setup of the environment

Below the necessary libraries are installed and loaded into the environment.

In [6]:
!pip install -q beautifulsoup4==4.13.4
!pip install -q docling==2.31.0

In [41]:
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup, Comment
import docling
import matplotlib.pyplot as plt

Below we mount a shared Google Drive folder as a data storage and define the base path of the folder that will be used in the runtime.

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
base_folder = '/content/drive/MyDrive/AdvGenAI'

## 2. Loading the raw data

We go through the subdirectories inside the data-folder. Inside those folders the individual html-files will be read and the content will be saved together with the information of the file-name and the path of the file (to store in which subfolder it was located).

In [12]:
# Definition of data folder
data_folder = os.path.join(base_folder, 'data')

In [ ]:
# List to hold the dictionaries
data = []

# Walk through all directories and subdirectories
for root, dirs, files in os.walk(data_folder):
    for file in files:
        if file.endswith('.html'):
            file_path = os.path.join(root, file)

            # Read the content of the HTML file
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()

            # Add a dictionary to the list
            data.append({
                'folder_path': root,
                'file_name': file,
                'full_path': file_path,
                'html_content': content
            })

# Convert to DataFrame
df = pd.DataFrame(data)

# Optionally save DataFrame, e.g. to CSV or pickle for later use
# df.to_csv('html_files_content.csv', index=False)
# df.to_pickle('html_files_content.pkl')

# Show first rows to verify
print(df.head())

In [49]:
pd.set_option('display.max_colwidth', 50)
df.head()

,folder_path,file_name,full_path,html_content
0,/content/drive/MyDrive/AdvGenAI/data/de_intern...,der-r-pionier1.html,/content/drive/MyDrive/AdvGenAI/data/de_intern...,"<div class=""text-image cq-dd-image"">\n<p>Währe..."
1,/content/drive/MyDrive/AdvGenAI/data/de_intern...,web-of-science-alles-neu-macht-der--januar-.html,/content/drive/MyDrive/AdvGenAI/data/de_intern...,"<div class=""text-image cq-dd-image"">\n<p><a cl..."
2,/content/drive/MyDrive/AdvGenAI/data/de_intern...,swiss-life-sciences-2014-experten-gesucht.html,/content/drive/MyDrive/AdvGenAI/data/de_intern...,"<div class=""text-image cq-dd-image"">\n<h2>Staf..."
3,/content/drive/MyDrive/AdvGenAI/data/de_intern...,mendeley-literaturverwaltung.html,/content/drive/MyDrive/AdvGenAI/data/de_intern...,"<div class=""text-image cq-dd-image"">\n<p><a cl..."
4,/content/drive/MyDrive/AdvGenAI/data/de_intern...,mehr-orientierung.html,/content/drive/MyDrive/AdvGenAI/data/de_intern...,"<div class=""text-image cq-dd-image"">\n<p>Das S..."


Below we compare the number of documents collected by the function into the Dataframe with a selection of all files in the data folder.

In the check 3 files were discovered that were not part of the dataframe. After inspection it was discovered that those are `.DS_Store`file, for which it makes sense that they were not included.

In [19]:
# Dataframe
print(f"Number of files in the DataFrame: {len(df)}")

Number of files in the DataFrame: 4390


In [20]:
# Files in Data folder
print(f"Number of files in the data folder:")
!find "$data_folder" -type f | wc -l

Number of files in the data folder:
4393


In [25]:
!find "$data_folder" -type f | sort > folder_files.txt
df['full_path'].sort_values().to_csv('df_files.txt', index=False, header=False)
!sort folder_files.txt -o folder_files.txt
!sort df_files.txt -o df_files.txt
!comm -23 folder_files.txt df_files.txt

/content/drive/MyDrive/AdvGenAI/data/de_internal/2013/.DS_Store
/content/drive/MyDrive/AdvGenAI/data/de_internal/2015/.DS_Store
/content/drive/MyDrive/AdvGenAI/data/de_internal/2024/.DS_Store


## 3. Parsing and cleaning the HTML files

In [31]:
def clean_html(html_content):
    """
    Extracts the title and main content from HTML while removing unnecessary elements.

    Args:
        html_content (str): The HTML content to clean.

    Returns:
        tuple: (title, cleaned_content) where:
            - title (str): The title of the HTML document
            - cleaned_content (str): The main content of the HTML document with tags removed
    """
    # Parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract the title
    title = ""
    if soup.title:
        title = soup.title.get_text(strip=True)

    # Create a copy of the soup for content extraction
    content_soup = BeautifulSoup(str(soup), 'html.parser')

    # Remove unwanted elements
    for element in content_soup(['script', 'style', 'header', 'footer', 'nav', 'iframe', 'meta', 'link']):
        element.decompose()

    # Remove comments - using 'string' instead of deprecated 'text'
    for comment in content_soup.find_all(string=lambda string: isinstance(string, Comment)):
        comment.extract()

    # Get the body content or full document if no body
    main_content = content_soup.body or content_soup

    # Get clean text with spacing between elements
    clean_text = main_content.get_text(separator=' ', strip=True)

    return title, clean_text

In [88]:
# create a subset of the dataframe for testing
df_test = df.sample(n=5).copy()

In [89]:
df_test['title'], df_test['clean_content'] = zip(*df_test['html_content'].apply(clean_html))

# Example usage with a pandas DataFrame:
# df['title'], df['clean_content'] = zip(*df['html_contents'].apply(clean_html))

In [90]:
pd.set_option('display.max_colwidth', 150)
df_test[['html_content', 'title', 'clean_content']]

,html_content,title,clean_content
1330,"<div class=""text-image cq-dd-image"">\n<p>Nass, kalt und ungemütlich ist es in Davos Mitte Juli. Auf 2000 Meter über Meer liegt neuer Schnee. Nicht...",,"Nass, kalt und ungemütlich ist es in Davos Mitte Juli. Auf 2000 Meter über Meer liegt neuer Schnee. Nichtsdestotrotz gehen ein Dutzend Studierende..."
1857,"<div class=""text-image cq-dd-image"">\n<figure class=""text-image__image text-image__image--large"">\n<a href=""https://covid19survey.ethz.ch/"">\n<img...",,Bereits nehmen über 3500 Personen am COVID-19-Monitoring teil. (Bild: BMI) Wie schnell breitet sich das Coronavirus in der Schweiz aus? In welchen...
2440,"<div class=""text-image cq-dd-image"">\n<p>In our newest blog post, we present the checklist ""<a class=""eth-link"" href=""http://bit.ly/rdmchecklist"">...",,"In our newest blog post, we present the checklist "" external page Data Management call_made "" (pdf, 281 kB) and show the advantages of a sound dat..."
3238,"<div class=""text-image cq-dd-image"">\n<p><b>Prof. Siegwart, the Federal Council has launched new National Centres of Competence in Research (NCCR)...",,"Prof. Siegwart, the Federal Council has launched new National Centres of Competence in Research (NCCR), whereby ETH Zurich is involved in four in ..."
250,"<div class=""text-image cq-dd-image"">\n<p>Der Masterplan für die Teilphase vom 3. bis 30. August zeigt, dass die ETH Zürich weiterhin sehr behutsam...",,"Der Masterplan für die Teilphase vom 3. bis 30. August zeigt, dass die ETH Zürich weiterhin sehr behutsam vorgehen will – Spielräume, wo möglich u..."


In [102]:
from IPython.display import HTML
import textwrap
for idx, row in df_test.iterrows():
    print(f"Row {idx}:")
    display(HTML(row['html_content']))
    # Display the cleaned text
    print("\nCleaned Text:\n")
    print(textwrap.fill(str(row['clean_content']), width=100))
    print(100 * "-")
    print("\n")

Row 1330:



Cleaned Text:

Nass, kalt und ungemütlich ist es in Davos Mitte Juli. Auf 2000 Meter über Meer liegt neuer Schnee.
Nichtsdestotrotz gehen ein Dutzend Studierende, Assistenten und drei Professoren des Departements
Umweltsystemwissenschaften (D-USYS) auf Exkursion. «Böden und Vegetation der Alpen» ist das Thema,
wobei sie das Programm dem Wetter anpassen mussten: Statt mit Pflanzen oberhalb der Waldgrenze
befassen sich die Studierenden mit Bodentypen und der Vegetation unterhalb davon: Fichtenwald auf
saurem, nährstoffarmem, sogenanntem Podsol-Boden, Bergföhrenwald auf Serpentinitgestein. Während die
Gruppe Geobotanik-Professor Matthias Baltisberger zuhört, haben viele Studierende ein Aha-Erlebnis:
die vielen Keimlinge einer Bergföhre, die neben dem Mutterbaum aus dem Serpentinitschutt
hervorschauen; der Gesteinsschutt, der vor wenigen Jahren von einem Murgang ins Tal verfrachtet
wurde und bei jedem Schritt unter den Füssen nachgibt, der aktuelle Dauerregen, der solche Murgänge
auslösen


Cleaned Text:

Bereits nehmen über 3500 Personen am COVID-19-Monitoring teil. (Bild: BMI) Wie schnell breitet sich
das Coronavirus in der Schweiz aus? In welchen Orten oder Regionen gibt es überdurchschnittlich
viele Neuinfektionen? Wo flacht die Kurve der Ansteckungen ab, wo steigt sie? Antworten auf diese
Fragen liefern derzeit vor allem die Coronavirus-Tests, aus denen hervorgeht, wie viele Personen in
der Schweiz positiv auf das Virus getestet worden sind. Viele Personen haben jedoch keine oder nur
wenig ausgeprägte Symptome. Sie werden nicht durch die Corona Tests erfasst. Biomedizininformatiker
der ETH Zürich haben nun ein Monitoring entworfen, bei dem alle Bewohnerinnen und Bewohner der
Schweiz mitmachen können – auch solche mit milden oder gar keinen Symptomen. Verstehen, wie sich das
Virus verbreitet «Unser Ziel ist es, besser zu verstehen und schneller zu erkennen, wie sich das
Coronavirus in der in der Schweizer Bevölkerung verbreitet», sagt Gunnar Rätsch, Professor für
Bio


Cleaned Text:

In our newest blog post, we present the checklist " external page Data Management call_made " (pdf,
281 kB) and show the advantages of a sound data management. Blog Innovation@ETH-Bibliothek Subscribe
to Newsletter Subscribe to the Newsletter for internal news Staffnet Info portal for employees with
the key information about what’s going on at ETH Zurich.
----------------------------------------------------------------------------------------------------


Row 3238:



Cleaned Text:

Prof. Siegwart, the Federal Council has launched new National Centres of Competence in Research
(NCCR), whereby ETH Zurich is involved in four in a leading or co-leading capacity. What is your
reaction as ETH Vice President Research and Corporate Relations? Roland Siegwart: I'm very happy.
This is a huge success for ETH Zurich. Our researchers put a great deal of energy and enthusiasm
into their applications. I'm all the more delighted that their visions and dreams are now coming
true. What do you think are the reasons for ETH's success in this NCCR series? All four NCCRs
supported by ETH, i.e. "Digital Fabrication", "RNA and Disease", "SwissMAP" and "Molecular Systems
Engineering", are based on convincing scientific visions and have the potential to have a long-term
impact on the Swiss research and innovation landscape. As NCCRs are selected in a competition, one
can justifiably say that scientific quality and excellence have been rewarded. Another major
contribution t


Cleaned Text:

Der Masterplan für die Teilphase vom 3. bis 30. August zeigt, dass die ETH Zürich weiterhin sehr
behutsam vorgehen will – Spielräume, wo möglich und Grenzen, wo nötig. Im Zentrum steht die
generelle Öffnung der Gebäude für ETH-Angehörige sowie für Besucherinnen und Besucher der
Bibliothek, der Ausstellungen und Sammlungen sowie für Handwerker und Gäste von ETH-Angehörigen. Es
gelten die gleichen Öffnungszeiten wie vor der Schliessung. Der allgemeinen Öffentlichkeit bleibt
der Zutritt aber vorderhand noch verwehrt. Gebäude ohne Prüfungen können auf begründeten Antrag von
Departementen oder Abteilungen geschlossen bleiben. «Der August steht ganz im Zeichen der
Sessionsprüfungen», erklärt Ueli Weidmann, Vizepräsident für Infrastruktur und Leiter der ETH-Covid-
Taskforce. Die grössten Prüfungen konzentrieren sich zwar auf wenige Gebäude, doch verteilen sich
die über 600 schriftlichen und 400 mündlichen Prüfungen auf viele weitere Örtlichkeiten. So ist die
grundsätzliche Geb